In [21]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from drawnow import drawnow

%matplotlib 



Using matplotlib backend: TkAgg


## problem 1 - Advection of scalar field over the sphere at an angle $\alpha$


### Constants in the problem 

In [2]:

ep = 6  ## optimal value of eps chosen for initial cosin bell condition refer pg 13/26 of the paper
alpha = np.pi/2  ## taken as flowing right over the poles: so as to compare with other methods, since error is max at poles
a = 6.37122e6  ## radius of earth
u0 = 2*np.pi*a/12  ## given in the pde qestion
R = a/3  ## defined in text - refer intial conditions 

### Node Distribution

In [3]:
# Load nodes  - node distribution based on electrostatic repulsion 
me1849 = np.loadtxt('me42.1849')
x = me1849[:,0]
y = me1849[:,1]
z = me1849[:,2]

sz = 10
plt.figure(1)
plt.scatter(x, y, s=sz, c='k', alpha=0.5)
plt.title('ME1849 nodes distribution on the surface of a unit sphere')
plt.show()

### Compute $r^2 = (x_j - x_k)^2 + (y_j - y_k)^2 + (z_j - z_k)^2$  

In [4]:
"""
This computes the euclidean distance between each node and all the other nodes ; 
Thus this is a symmetric matrix of size is 1849 * 1849 with all the diagonal elements = 0
"""
nodes = np.column_stack((x,y,z))
rd2 = np.zeros((len(nodes), len(nodes)))
for j in range(3):
    xd1 = nodes[:,j].reshape(-1, 1)
    xd2 = xd1.T
    rd2 += (xd1 - xd2)**2
    
    
print(rd2.max())    

3.999998257778941


### Set up a 2D surface grids in $(\theta,\phi)$ for computing B

In [5]:
# In text theta = larutude ; lamda = longitude ; here theta = latitude phi = longitude
theta = np.arctan2(z, np.sqrt(x**2 + y**2))  ## so for each x,y,z node point so there are 1849 values of theta that is latitude value taken from the equator 
phi = np.arctan2(y, x)  ## similarly there are 1849 values of phi i.e. the values of longitude
tn = np.tile(theta, (len(x), 1))    ## this makes a 1849 * 1849 matrix where each row is the same and looks exactly like theta ; thus all the columns will have the same value 
tc = tn.T  ## this is just the transpose of tn; therefore all the rows have one value of theta repeated 1849 times
pn = np.tile(phi, (len(phi), 1))   ## similar to tn but phi values 
pc = pn.T   ## similr to tc and has all rows equal to one ohi value repeated 1849 times

### Compute differentiation matrix D  

In [20]:
#exactly as given in the paper only theta i,j = tn,tc and lambda i,j = pn,pc
#np.cos(alpha)*np.cos(tn)*np.cos(tc)*np.sin(pn-pc) +np.sin(alpha)*(np.cos(tn)*np.cos(pn)*np.sin(tc) - np.cos(tc)*np.cos(pc)*np.sin(tn))
#((((np.cos(tc)*np.sin(pn-pc))*(np.cos(tn)*np.cos(alpha)+np.sin(tn)*np.cos(pn)*np.sin(alpha)))/(np.cos(tn)))+(-np.sin(pn)*np.sin(alpha)) * (np.cos(tc)*np.sin(tn)*np.cos(pn-pc)  -  np.sin(tc)*np.cos(tn)))

t = tn
tn = tc
tc = t
ep = 6

B = np.cos(alpha)*np.cos(tn)*np.cos(tc)*np.sin(pn-pc) +np.sin(alpha)*(np.cos(tn)*np.cos(pn)*np.sin(tc) - np.cos(tc)*np.cos(pc)*np.sin(tn))
B = 2*(ep**2)*(u0/a)*B*np.exp(-(ep**2)*rd2)

print(B[57,6])

2.157053377021666


In [22]:
A = np.exp(-ep**2*rd2)
A_inv = np.linalg.inv(A)
D = np.matmul(B,A_inv)
print(D.max())


print(A)

6.258946083907489
[[1.00000000e+00 4.32438611e-11 4.06274111e-37 ... 3.44199850e-56
  1.12432753e-59 4.46168361e-63]
 [4.32438611e-11 1.00000000e+00 2.44372734e-27 ... 2.56027783e-39
  1.06867346e-39 2.97851096e-50]
 [4.06274111e-37 2.44372734e-27 1.00000000e+00 ... 1.63023287e-38
  9.81196148e-22 3.79662147e-26]
 ...
 [3.44199850e-56 2.56027783e-39 1.63023287e-38 ... 1.00000000e+00
  2.02374037e-05 2.00444815e-06]
 [1.12432753e-59 1.06867346e-39 9.81196148e-22 ... 2.02374037e-05
  1.00000000e+00 6.45153036e-03]
 [4.46168361e-63 2.97851096e-50 3.79662147e-26 ... 2.00444815e-06
  6.45153036e-03 1.00000000e+00]]


### Initial Condition = Cosine bell

In [23]:
# Initial Condition Cosine Bell
r = a*np.arccos(np.cos(theta)*np.cos(phi))    ## as defined in text
h = 1000/2*(1 + np.cos(np.pi*r/R))   ## as defined in text 
h[r >= R] = 0  ## as defined in text

## making sure there are non zero values of h initially print(np.nonzero(h))
h_initial = h

print(h.size)

1849


### 3D plot to compare with text

In [24]:
fig = plt.figure(2)
ax = fig.add_subplot(111, projection='3d')
# Plot initial condition
ax.scatter(phi, theta, h, c=h, cmap='viridis')
# Set axis labels
ax.set_xlabel('$\lambda$')
ax.set_ylabel(r'$\theta$')
ax.set_zlabel('h')
plt.title('Initial Condition - Cosine Bell')
plt.show()

### Time-Stepping - 4th Order RK - 50 minute time step

In [25]:
dt = 12/288*5/6 ## Time-Step for 12 days revolution (50 imnutes in terms of days)
h_rk = h

for nt in range(2, 1*288*6//5 +1):
    d1 = dt*D.dot(h_rk)
    d2 = dt*D.dot(h_rk + 0.5*d1)
    d3 = dt*D.dot(h_rk + 0.5*d2)
    d4 = dt*D.dot(h_rk + d3)
    h_rk = h_rk + 1/6*(d1 + 2*d2 + 2*d3 + d4)

### 3D plot for better visualization

In [26]:
fig = plt.figure(8)
ax = fig.add_subplot(111, projection='3d')
# Plot initial condition
ax.scatter(phi, theta, h_rk, c=h, cmap='viridis')
# Set axis labels
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\theta$')
ax.set_zlabel('h')
plt.title('Cosine bell,RBF solution after 12 days')
plt.show()

### Comparison with analytical solution 

In [27]:
fig = plt.figure(9)
ax = fig.add_subplot(111, projection='3d')
# Plot initial condition
ax.scatter(phi, theta, h_rk, c="y", label = "numerical solution")
ax.scatter(phi, theta, h,c = "b", label = "analytical solution")
# Set axis labels
ax.set_xlabel('phi')
ax.set_ylabel('theta')
ax.set_zlabel('h')
plt.legend()
plt.show()

### Initial Condition = Exceptionally steep gaussian profile


In [28]:
ep = 6

B2 = np.cos(alpha)*np.cos(tn)*np.cos(tc)*np.sin(pn-pc) +np.sin(alpha)*(np.cos(tn)*np.cos(pn)*np.sin(tc) - np.cos(tc)*np.cos(pc)*np.sin(tn))
B2 = 2*(ep**2)*(u0/a)*B2*np.exp(-(ep**2)*rd2)

A2 = np.exp(-ep**2*rd2)
A_inv2 = np.linalg.inv(A2)

D2 = np.matmul(B2,A_inv2)
print(D2.max())

6.258946083907489


In [29]:
# Initial Condition Cosine Bell
r = a*np.arccos(np.cos(theta)*np.cos(phi))    ## as defined in text
h_gaussian = 1000*np.exp(-((1*2.25*r/R)**2))   ## as defined in text 

## making sure there are non zero values of h initially print(np.nonzero(h))
h_initial = h_gaussian

print(h_gaussian.size)

1849


### 3D plot to compare with text

In [30]:

fig = plt.figure(11)
ax = fig.add_subplot(111, projection='3d')
# Plot initial condition
cax = ax.scatter(phi, theta, h_gaussian, c=h, cmap='cividis')
# Set axis labels
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\theta$')
ax.set_zlabel('h')
plt.title('Initial Condition - Gaussian Bell')
plt.show()

### Time-Stepping - 4th Order RK - 50 minute time step

In [31]:
dt = 12/288*5/6 ## Time-Step for 12 days revolution (50 imnutes in terms of days)
h_rk2 = h_gaussian

for nt in range(2, 1*288*6//5 + 1):
    d1 = dt*D2.dot(h_rk2)
    d2 = dt*D2.dot(h_rk2 + 0.5*d1)
    d3 = dt*D2.dot(h_rk2 + 0.5*d2)
    d4 = dt*D2.dot(h_rk2 + d3)
    h_rk2 = h_rk2 + 1/6*(d1 + 2*d2 + 2*d3 + d4)
      
print(h_gaussian.max()) 
print(h_rk2.max())

984.188215229473
991.570869600357


### 3D plot for better visualization

In [32]:

fig = plt.figure(12)
ax = fig.add_subplot(111, projection='3d')
# Plot initial condition
ax.scatter(phi, theta, h_rk2, c=h, cmap='cividis')
# Set axis labels
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\theta$')
ax.set_zlabel('h')
plt.title('Gaussian bell, RBF solution after 12 days')
plt.savefig('gs.png')
plt.show()

In [34]:
fig = plt.figure(13)
ax = fig.add_subplot(111, projection='3d')
# Plot initial condition
ax.scatter(phi, theta, h_gaussian - h_rk2, c= 'mediumblue')
ax.scatter(phi, theta, h - h_rk, c= 'orangered')
# Set axis labels
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\theta$')
ax.set_zlabel('h')
plt.title('Error (exact - numerical)')
plt.savefig('error.png')
plt.show()

### Comparison with analytical solution 

In [47]:
# Data for the box plot
data = [h_gaussian - h_rk2, h - h_rk]

# Create a figure and axes
fig, ax = plt.subplots()

# Create the box plot
ax.boxplot(data)

# Set the labels
ax.set_xticklabels(['Gaussia: Exact - Numerical', 'Cosine : Exact - Numerical'])
ax.set_ylabel('Error')

# Set the title
plt.title('Box Plot of Errors')

# Display the plot
plt.show()

In [43]:
import matplotlib.pyplot as plt

# Data for the histogram
data = h_gaussian - h_rk2

# Create a figure and axes
fig, ax = plt.subplots()

# Create the histogram
ax.hist(data, bins=35)

# Set the labels
ax.set_xlabel('Error')
ax.set_ylabel('Frequency')

# Set the title
plt.title('Histogram of Errors')

# Display the plot
plt.show()


In [51]:
# Data for the contour plot
X = phi
Y = theta
Z = h - h_rk  # Error data

# Create a figure and axes
fig, ax = plt.subplots()

# Create the contour plot
contour = ax.tricontourf(X, Y, Z, cmap='magma')

# Add a colorbar
cbar = plt.colorbar(contour)

# Set the labels
ax.set_xlabel(r'$\lambda$')
ax.set_ylabel(r'$\theta$')
cbar.set_label('Error')

# Set the title
plt.title('Contour Plot of Errors')

# Display the plot
plt.show()

In [52]:
import matplotlib.pyplot as plt

# Data for the histograms
data1 = h - h_rk
data2 = h_gaussian - h_rk2

# Create a figure and axes
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Create the histograms
ax1.hist(data1, bins=10)
ax2.hist(data2, bins=10)

# Set the labels for the first subplot
ax1.set_xlabel('Error')
ax1.set_ylabel('Frequency')
ax1.set_title('Cosine bell')

# Set the labels for the second subplot
ax2.set_xlabel('Error')
ax2.set_ylabel('Frequency')
ax2.set_title('Gaussian bell')

# Adjust the spacing between subplots
plt.tight_layout()

# Display the plot
#plt.title('Error (exact - Numerical)')
plt.show()

invalid command name "139624647421504delayed_destroy"
    while executing
"139624647421504delayed_destroy"
    ("after" script)
invalid command name "139624684588672delayed_destroy"
    while executing
"139624684588672delayed_destroy"
    ("after" script)
invalid command name "139624650958784delayed_destroy"
    while executing
"139624650958784delayed_destroy"
    ("after" script)
invalid command name "139624647026176delayed_destroy"
    while executing
"139624647026176delayed_destroy"
    ("after" script)


In [35]:
fig = plt.figure(14)
ax = fig.add_subplot(111, projection='3d')
# Plot initial condition
ax.scatter(phi, theta, h_rk2, c="y", label = "numerical solution for gaussian")
ax.scatter(phi, theta, h_gaussian,c = "black", label = "analytical solution for gaussin")
ax.scatter(phi, theta, h_rk, c="g", label = "numerical solution for cosine bell")
ax.scatter(phi, theta, h,c = "r", label = "analytical solution for cosine bell")
# Set axis labels
ax.set_xlabel('phi')
ax.set_ylabel('theta')
ax.set_zlabel('h')
plt.legend()
plt.show()